# Single Layer Perceptron

Let's explore _perceptron_, a single-layer (inputs are fed directly to outputs via a series of weights) feedforward (connections between nodes do not form cycles) neural network. It can classify linearly separable patterns.

Formally, it is defined as $y = f(w^Tx - b)$, where $w$ is a vector of weights, $x$ is the vector of inputs, $||w|| = ||x||$ (by definition of multiplication), $b$ is the activation threshold (bias), and $f(x)$ is the activation function.

The learning algorithm, given a set of input vectors $\{x_1, ..., x_N\}$ and a corresponding set of desired labels $\{y_1, ..., y_N\}$, finds such a vector of weights $w$ that classifies all examples correctly, starting from some initial (random) vector $w$. It takes inputs one by one, calculates the label, and proceeds to the next input if the classification is correct. Otherwise, it calculates a new $w_{i+1} = w_i \pm x$ ($+$ in case of a false positive, $-$ in case of a false negative). The algorithm reiterates through inputs until none of them are misclassified.

As an example, let's proceed to implement a couple of simple binary functions as perceptrons. The activation function will be $f(x) = \begin{cases}1 \text{ if } x \geq 0,\\ 0 \text{ otherwise}\end{cases}$

In [1]:
import Numeric.LinearAlgebra (Vector(..), vector, rand, flatten, (<.>))
import Control.Monad.Writer

-- Perceptron applies the activation function to a dot product of 
perceptron :: Vector Double -> Vector Double -> Double -> Double
perceptron input weights bias
  | input <.> weights - bias > 0 = 1.0
  | otherwise = 0.0

-- The learning function accepts a list of (input vector, corresponding label) tuples
-- and an initial vector of weights.
learn :: [(Vector Double, Double)] -> Vector Double -> Writer String (Vector Double)
learn labelledInputs initialWeights = go labelledInputs initialWeights False
  where
    -- The boolean indicates whether we should perform another iteration on the input set.
    go :: [(Vector Double, Double)] -> Vector Double -> Bool -> Writer String (Vector Double)
    go [] ws False = 
      tell "All inputs have been classified correctly, halting" >> pure ws
    go [] ws True =
      tell "Some of the inputs have been misclassified, performing another iteration\n" >>
      go labelledInputs ws False
    go ((i, l):rest) ws iter
      | (perceptron i ws 1.0) == l =
        tell (show i ++ " is classified correctly\n") >>
        go rest ws iter
      | (perceptron i ws 1.0) > l =
        tell (show i ++ " yielded a false positive, decrementing the weights\n") >>
        go rest (ws - i) True
      | (perceptron i ws 1.0) < l =
        tell (show i ++ " yielded a false negative, incrementing the weights\n") >>
        go rest (ws + i) True
        
-- A helper function to print training results
runSLP :: [Vector Double] -> [Double] -> IO ()
runSLP inputs labels =
  let labelledInputs = zip inputs labels
      (weights, log) = runWriter $ learn labelledInputs (vector [0.0, 0.0])
  in putStrLn ("Training an SLP on " ++ show labelledInputs) >>
     putStrLn log >>
     putStrLn ("Final weights are " ++ show weights)

## Boolean AND

In [2]:
inputs = [vector [a, b] | a <- [0, 1], b <- [0, 1]]

labels = [binaryAnd a b | a <- [0, 1], b <- [0, 1]]
  where binaryAnd 1 1 = 1.0
        binaryAnd _ _ = 0.0

runSLP inputs labels

Training an SLP on [([0.0,0.0],0.0),([0.0,1.0],0.0),([1.0,0.0],0.0),([1.0,1.0],1.0)]
[0.0,0.0] is classified correctly
[0.0,1.0] is classified correctly
[1.0,0.0] is classified correctly
[1.0,1.0] yielded a false negative, incrementing the weights
Some of the inputs have been misclassified, performing another iteration
[0.0,0.0] is classified correctly
[0.0,1.0] is classified correctly
[1.0,0.0] is classified correctly
[1.0,1.0] is classified correctly
All inputs have been classified correctly, halting
Final weights are [1.0,1.0]

## Boolean OR

In [3]:
labels = [binaryOr a b | a <- [0, 1], b <- [0, 1]]
  where binaryOr 0 0 = 0.0
        binaryOr _ _ = 1.0

runSLP inputs labels

Training an SLP on [([0.0,0.0],0.0),([0.0,1.0],1.0),([1.0,0.0],1.0),([1.0,1.0],1.0)]
[0.0,0.0] is classified correctly
[0.0,1.0] yielded a false negative, incrementing the weights
[1.0,0.0] yielded a false negative, incrementing the weights
[1.0,1.0] is classified correctly
Some of the inputs have been misclassified, performing another iteration
[0.0,0.0] is classified correctly
[0.0,1.0] yielded a false negative, incrementing the weights
[1.0,0.0] yielded a false negative, incrementing the weights
[1.0,1.0] is classified correctly
Some of the inputs have been misclassified, performing another iteration
[0.0,0.0] is classified correctly
[0.0,1.0] is classified correctly
[1.0,0.0] is classified correctly
[1.0,1.0] is classified correctly
All inputs have been classified correctly, halting
Final weights are [2.0,2.0]